In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy
# 1923 earliest year

In [ ]:
# read in new merged gbif_obis data
new_path = input("Enter the file path: ")

new = pd.read_parquet(new_path)


In [ ]:
# read in old species data
old_path = input("Enter the file path: ")

# read in the old species data in json format
old = pd.read_json(old_path)

In [ ]:
old


In [ ]:
species = old["scientific_name"]

In [ ]:
# check for strange x at beginning of species names and remove if necessary

# old = old[
#     ~old.scientific_name.isin(["× Heucherella", "×Elyhordeum schaackianum", "×Elyleymus aleuticus"])
# ]

In [ ]:
# check for species names that longer than 2 words and remove if necessary
species_values = np.array([len(word.split()) for word in species])

# find the max value in species_values
max_value = np.max(species_values)


In [ ]:
# if value at index i in species_values is greater than 2, add the value at the same index in species_unique to a new list
species_2 = []
for i in range(len(species_values)):
    if species_values[i] > 2:
        species_2.append(species[i])


In [ ]:
# remove values from species that are greater than 2

for i in range(len(species_2)):
    old = old[old.scientific_name != species_2[i]]

In [ ]:
del species_2, species_values, max_value, i, species


In [ ]:
merged = pd.merge(new, old, how="left", on=["scientific_name"], indicator=True)

In [ ]:
# keep columns with less missing values

kingdom_x_missing = merged["kingdom_x"].isna().sum()

kingdom_y_missing = merged["kingdom_y"].isna().sum()

# drop the kingdom column from merged that contains more missing values
if kingdom_x_missing > kingdom_y_missing:
    merged = merged.drop(["kingdom_x"], axis=1)
    merged.rename(columns={"kingdom_y": "kingdom"}, inplace=True)
else:
    merged = merged.drop(["kingdom_y"], axis=1)
    merged.rename(columns={"kingdom_x": "kingdom"}, inplace=True)

phylum_x_missing = merged["phylum_x"].isna().sum()

phylum_y_missing = merged["phylum_y"].isna().sum()

# drop the phylum column from merged that contains more missing values

if phylum_x_missing > phylum_y_missing:
    merged = merged.drop(["phylum_x"], axis=1)
    merged.rename(columns={"phylum_y": "phylum"}, inplace=True)
else:
    merged = merged.drop(["phylum_y"], axis=1)
    merged.rename(columns={"phylum_x": "phylum"}, inplace=True)

# find sum of missing values in merged['class_x']
class_x_missing = merged["class_x"].isna().sum()

class_y_missing = merged["class_y"].isna().sum()

# drop the class column from merged that contains more missing values
if class_x_missing > class_y_missing:
    merged = merged.drop(["class_x"], axis=1)
    merged.rename(columns={"class_y": "class"}, inplace=True)
else:
    merged = merged.drop(["class_y"], axis=1)
    merged.rename(columns={"class_x": "class"}, inplace=True)

# check
class_missing = merged["class"].isna().sum()

order_x_missing = merged["order_x"].isna().sum()

order_y_missing = merged["order_y"].isna().sum()

# drop the order column from merged that contains more missing values

if order_x_missing > order_y_missing:
    merged = merged.drop(["order_x"], axis=1)
    merged.rename(columns={"order_y": "order"}, inplace=True)
else:
    merged = merged.drop(["order_y"], axis=1)
    merged.rename(columns={"order_x": "order"}, inplace=True)

family_x_missing = merged["family_x"].isna().sum()

family_y_missing = merged["family_y"].isna().sum()

# drop the family column from merged that contains more missing values

if family_x_missing > family_y_missing:
    merged = merged.drop(["family_x"], axis=1)
    merged.rename(columns={"family_y": "family"}, inplace=True)
else:
    merged = merged.drop(["family_y"], axis=1)
    merged.rename(columns={"family_x": "family"}, inplace=True)


genus_x_missing = merged["genus_x"].isna().sum()

genus_y_missing = merged["genus_y"].isna().sum()

# drop the genus column from merged that contains more missing values

if genus_x_missing > genus_y_missing:
    merged = merged.drop(["genus_x"], axis=1)
    merged.rename(columns={"genus_y": "genus"}, inplace=True)
else:
    merged = merged.drop(["genus_y"], axis=1)
    merged.rename(columns={"genus_x": "genus"}, inplace=True)

In [ ]:
del (
    genus_y_missing,
    genus_x_missing,
    family_y_missing,
    family_x_missing,
    order_y_missing,
    order_x_missing,
    class_y_missing,
    class_x_missing,
    phylum_y_missing,
    phylum_x_missing,
    kingdom_y_missing,
    kingdom_x_missing,
    class_missing,
)

In [ ]:
merged


In [ ]:
merged = merged.drop(["unique_id_y", "rights_y", "_merge"], axis=1)

merged.rename(columns={"unique_id_x": "unique_id",
              "rights_x": "rights"}, inplace=True)


In [ ]:
merged


In [ ]:
merged = merged[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [32]:
# read cleaned df into a parquet file where user can input the file path
df_merge_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
merged.to_parquet(df_merge_path)
